In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix,classification_report
from wordcloud import WordCloud 
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Flatten, Conv1D, Dropout, Activation
from keras.preprocessing.text import Tokenizer

In [3]:
df_train= pd.read_csv("train_E6oV3lV.csv")
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
s=df_train.shape
s

(31962, 3)

In [5]:
df_test= pd.read_csv("test_tweets_anuFYb8.csv")
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [6]:
n=  df_test.shape
n= n[0]
n

17197

In [7]:
df=  pd.concat([df_train, df_test])
df.shape

C:\Users\91758\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(49159, 3)

In [8]:
#remove username starting with @
df['clean_tweet']= df['tweet'].replace(r'@\w+','',regex=True)

In [9]:
def decontracted(phrase):
    phrase = re.sub(r"n\'t", "n not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

#remove apostrphe from the word
df['clean_tweet']= [decontracted(i) for i in df['clean_tweet']]

In [10]:
#remove RT(retweet) 
df['clean_tweet'].replace('RT','',regex=True, inplace= True)

# Removing the links present in our clean text 
df['clean_tweet'].replace('((www\.[^\s]+)|(https?://[^\s]+))', '\0', inplace=True, regex=True)

# Removing special characters, numbers, punctuations as none of them would add any value while making sentiment analysis
df['clean_tweet'].replace('[^a-zAA-Z]+', ' ', inplace=True, regex=True)

#Removing the word 'hmm' and it's variants
df['clean_tweet'] = df['clean_tweet'].apply(lambda x : ' '.join([word for word in x.split() if not word == 'h(m)+' ]))

# Converting all the clean_text column into lower case
df['clean_tweet'] = df['clean_tweet'].str.lower()

In [11]:
#Code for removing slang words
d = {'luv':'love','wud':'would','lyk':'like','wateva':'whatever','ttyl':'talk to you later',
               'kul':'cool','fyn':'fine','omg':'oh my god!','fam':'family','bruh':'brother',
               'cud':'could','fud':'food', 'u':'you'
    } ## Need a huge dictionary
words = "I luv myself"
words = words.split()
df['clean_tweet'] = df['clean_tweet'].apply(lambda x : ' '.join(d[word] if word in d else word for word in x.split()))

In [12]:
# TOKENIZATION
# Splitting each row of clean_text column into list of words
df['clean_tweet'] = df['clean_tweet'].str.split()

# Getting list of stopwords present in the nltk library
stop_words = stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')

# Removing the stopwords from every row of the column 'clean_text'
df['clean_tweet'] = df['clean_tweet'].apply(lambda x:[item for item in x if item not in stop_words])

# Doing Lemmatization through WordNetLemmatizer
w_n_lemmatizer = WordNetLemmatizer()
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: [w_n_lemmatizer.lemmatize(item) for item in x])

# Joining all the tokens into a string
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: ' '.join(x))

df['clean_tweet']

0        father dysfunctional selfish drag kid dysfunct...
1        thanks lyft credit not use cause not offer whe...
2                                           bihday majesty
3                                  model love take time ur
4                            factsguide society motivation
5        huge fan fare big talking leave chaos pay disp...
6                                   camping tomorrow danny
7        next school year year exam not think school ex...
8        love land allin cavs champion cleveland clevel...
9                                               welcome gr
10       ireland consumer price index mom climbed previ...
11       selfish orlando standwithorlando pulseshooting...
12                      get see daddy today day gettingfed
13       cnn call michigan middle school build wall cha...
14       no comment australia opkillingbay seashepherd ...
15            ouch junior angry got junior yugyoem oh god!
16                        thankful paner thankful positi

In [17]:
sv= int(s[0]*0.2)
sb= s[0]-sv
sb

25570

In [20]:
train= df[:sb]
val= df[sb:-n]
test= df[-n:]

In [72]:
VOCAB_SIZE = 20000
MAX_LEN = 90


tweet_tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tweet_tokenizer.fit_on_texts(train['clean_tweet'].values)

x_train_seq = tweet_tokenizer.texts_to_sequences(train['clean_tweet'].values)
x_val_seq = tweet_tokenizer.texts_to_sequences(val['clean_tweet'].values)

x_train = sequence.pad_sequences(x_train_seq, maxlen=MAX_LEN, padding="post", value=0)
x_val = sequence.pad_sequences(x_val_seq, maxlen=MAX_LEN, padding="post", value=0)

y_train, y_val = train['label'].values, val['label'].values

print('First sample before preprocessing: \n', train['clean_tweet'].values[0], '\n')
print('First sample after preprocessing: \n', x_train[0])

First sample before preprocessing: 
 father dysfunctional selfish drag kid dysfunction run 

First sample after preprocessing: 
 [   14 11780  2530  2998   120  6011   339     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]


In [76]:
# Model Parameters - You can play with these

NUM_FILTERS = 250
KERNEL_SIZE = 3
HIDDEN_DIMS = 250
EMBEDDING_DIM = 40
BATCH_SIZE = 32
EPOCHS = 3

In [77]:
# CNN Model
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into EMBEDDING_DIM dimensions
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn NUM_FILTERS filters
model.add(Conv1D(NUM_FILTERS,
                 KERNEL_SIZE,
                 padding='valid',
                 activation='relu',
                 strides=1))

# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(HIDDEN_DIMS))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 90, 40)            800000    
_________________________________________________________________
dropout_9 (Dropout)          (None, 90, 40)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 88, 250)           30250     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 250)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_10 (Dropout)         (None, 250)               0         
_________________________________________________________________
activation_9 (Activation)    (None, 250

In [78]:
model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.1,
          verbose=2)

# Evaluate the model
score, acc = model.evaluate(x_val, y_val, batch_size=BATCH_SIZE)
print('\nAccuracy: ', acc*100)

pred = model.predict_classes(x_val)

C:\Users\91758\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 23013 samples, validate on 2557 samples
Epoch 1/3
 - 33s - loss: 0.1806 - accuracy: 0.9413 - val_loss: 0.1241 - val_accuracy: 0.9593
Epoch 2/3
 - 30s - loss: 0.0763 - accuracy: 0.9738 - val_loss: 0.1185 - val_accuracy: 0.9613
Epoch 3/3
 - 31s - loss: 0.0350 - accuracy: 0.9883 - val_loss: 0.1723 - val_accuracy: 0.9562
6392/6392 [==============================] - 1s 221us/step

Accuracy:  96.16708159446716


In [79]:
cnf_matrix = confusion_matrix(pred, y_val)
cnf_matrix

array([[5862,  158],
       [  87,  285]], dtype=int64)

In [80]:
#test data
x_test_seq = tweet_tokenizer.texts_to_sequences(test['clean_tweet'].values)

x_test = sequence.pad_sequences(x_test_seq, maxlen=MAX_LEN, padding="post", value=0)

In [81]:
pred1 = model.predict_classes(x_test)

In [82]:
pred1

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [83]:
l=[]
for sublist in pred1:
    for item in sublist:
        l.append(item)

In [84]:
l

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [85]:
p= pd.Series(l)

In [86]:
a= pd.concat([test['id'], p], axis=1)
a.columns=['id', 'label']

In [87]:
a.to_csv("Deep_learning_solution.csv", index= False)